In [1]:
import invisible_cities.io.rwf_io as rwfio

In [2]:
import sys
import os
import shutil
import functools
import time
import glob

%matplotlib inline
print("Running on ", time.asctime())

import numpy             as np
import tables            as tb
import matplotlib.pyplot as plt
from   matplotlib.colors import LogNorm
import pandas as pd

Running on  Thu Apr 11 15:27:01 2019


In [3]:
from matplotlib import pylab as plt

In [4]:
import numpy  as np
import tables as tb
import pandas as pd

from invisible_cities.types.ic_types import minmax
from invisible_cities.database       import load_db

from invisible_cities.core.system_of_units_c import units

from invisible_cities.reco                  import tbl_functions        as tbl
from invisible_cities.reco                  import  peak_functions      as pkf
from invisible_cities.core.random_sampling  import NoiseSampler         as SiPMsNoiseSampler
from invisible_cities.io  .        pmaps_io import          pmap_writer
from invisible_cities.io.        mcinfo_io  import       mc_info_writer
from invisible_cities.io  .run_and_event_io import run_and_event_writer
from invisible_cities.io  .trigger_io       import       trigger_writer
from invisible_cities.io  .event_filter_io  import  event_filter_writer

from invisible_cities.dataflow            import dataflow as fl
from invisible_cities.dataflow.dataflow   import push
from invisible_cities.dataflow.dataflow   import pipe
from invisible_cities.dataflow.dataflow   import sink

from invisible_cities.cities.components import city
from invisible_cities.cities.components import print_every
from invisible_cities.cities.components import deconv_pmt
from invisible_cities.cities.components import calibrate_pmts
from invisible_cities.cities.components import calibrate_sipms
from invisible_cities.cities.components import zero_suppress_wfs
from invisible_cities.cities.components import WfType
from invisible_cities.cities.components import wf_from_files
# dorothea
from invisible_cities.cities.components      import peak_classifier
from invisible_cities.filters.s1s2_filter    import S12Selector
from invisible_cities.cities.components             import peak_classifier

from   invisible_cities.cities.irene       import build_pmap
import invisible_cities.reco.dst_functions as dstf
import invisible_cities.io.dst_io          as dsts

from invisible_cities.icaro.hst_functions  import hist2d, hist2d_profile
from invisible_cities.core.fit_functions   import  *

import invisible_cities.core.fit_functions as fitf

## Open Dorothea Kr file:

In [5]:
demopp  = os.environ['ICTDIR'] + '/invisible_cities/database/localdb.DEMOPPDB.sqlite3'

In [6]:
demopp

'/Users/neus/IC/invisible_cities/database/localdb.DEMOPPDB.sqlite3'

In [7]:
sipm = load_db.DataSiPM(load_db.DetDB.demopp,533)

In [8]:
demopp

'/Users/neus/IC/invisible_cities/database/localdb.DEMOPPDB.sqlite3'

### Open files:

In [9]:
path = '/Users/neus/current-work/data/533/kdst/trigger1/'

In [10]:
files_all = [path+f for f in os.listdir(path) if os.path.isfile( os.path.join(path, f) )]

In [ ]:
d = dsts.load_dsts(files_all, "DST", "Events")

In [ ]:
len(d),type(d)

In [ ]:
d.dtypes.index

In [ ]:
d.head()

In [ ]:
d.tail()

## Selection:

In [ ]:
sel = (d.nS1 == 1) & (d.Z<350) 
sel2 = (d.nS1 == 1) & (d.nS2 == 1) & (d.Z < 350)  
sel3 = (d.nS1 == 1) & (d.nS2 == 1) & (d.Z < 300) & (d.Z > 20)  & (d.R<60)

In [ ]:
fig = plt.figure(figsize = (15,25))

ax2 = fig.add_subplot(6,3,1)
plt.hist(d.S2e,     100,[0,10000])
plt.hist(d.S2e[sel],100,[0,10000])
plt.xlabel('S2e')

ax2 = fig.add_subplot(6,3,2)
plt.hist(d.S2e,     100,[0,10000])
plt.hist(d.S2e[sel],100,[0,10000])
plt.xlabel('S2e')

ax2 = fig.add_subplot(6,3,3)
plt.hist(d.S2e[sel],100,[4000,7000])
plt.xlabel('S2e')

plt.show()

In [ ]:
fig = plt.figure(figsize = (11,13))

ax2 = fig.add_subplot(3,2,6)
#plt.hist(d.S2e, 100, [4000,7000])
plt.hist(d.S2e[sel], 100, [4000,7000])
plt.hist(d.S2e[sel2], 100, [4000,7000])
plt.hist(d.S2e[sel3], 100, [4000,7000])
plt.xlabel('S2e')

ax2 = fig.add_subplot(3,2,1)
plt.hist2d(d.Z[sel3], d.S2e[sel3],[100,100],[[0,350],[2500,7000]])
plt.xlabel('Z')
plt.ylabel('S2e')
plt.colorbar()

ax2 = fig.add_subplot(3,2,2)
plt.hist2d(d.Z[sel3], d.S1e[sel3],[100,100],[[0,350],[0,80]])
plt.xlabel('Z')
plt.ylabel('S1e')
plt.colorbar()

ax2 = fig.add_subplot(3,2,1)
plt.hist2d(d.R[sel3], d.S2e[sel3],[100,100], [[0,80],[2500, 7000]])
plt.xlabel('R')
plt.ylabel('S2e')
plt.colorbar()

plt.show()

In [ ]:
fig = plt.figure(figsize = (11,13))

ax2 = fig.add_subplot(3,2,1)
plt.hist(d.S2e[sel3],100,[4000,7000])
plt.xlabel('S2e')

ax2 = fig.add_subplot(3,2,2)
plt.hist2d(d.Z[sel3], d.S2e[sel3],[50,50],[[0,350],[5000,6500]])
plt.xlabel('Z')
plt.ylabel('S2e')
plt.colorbar()

ax2 = fig.add_subplot(3,2,3)
plt.hist2d(d.Z[sel3], d.S1e[sel3],[50,50],[[0,350],[0,20]])
plt.xlabel('Z')
plt.ylabel('S1e')
plt.colorbar()

ax2 = fig.add_subplot(3,2,4)
plt.hist2d(d.S1e[sel3], d.S2e[sel3], 100, [[0,30], [3000,6500]])
plt.xlabel('S1e')
plt.ylabel('S2e')
plt.colorbar()

ax2 = fig.add_subplot(3,2,5)
plt.hist2d(d.Nsipm[sel3], d.S2e[sel3], [50, 100], [[0,50],[3000,6500]])
plt.xlabel('Nsipm')
plt.ylabel('S2e')
plt.colorbar()

ax2 = fig.add_subplot(3,2,6)
plt.hist2d(d.Nsipm[sel3],d.S1e[sel3], [20, 100], [[0,20],[0,40]])
plt.xlabel('Nsipm')
plt.ylabel('S1e')
plt.colorbar()

plt.show()

In [ ]:
fig = plt.figure(figsize = (11,13))

ax2 = fig.add_subplot(3,2,3)
plt.hist(d.Z[sel3],100)
#plt.yscale('log', nonposy='clip')
plt.xlabel('Z')

ax2 = fig.add_subplot(3,2,4)
plt.hist(d.X[sel3],100)
#plt.yscale('log', nonposy='clip')
plt.xlabel('X')

ax2 = fig.add_subplot(3,2,5)
plt.hist(d.Y[sel3],100)
#plt.yscale('log', nonposy='clip')
plt.xlabel('Y')

ax2 = fig.add_subplot(3,2,3)
plt.hist2d(d.X[sel3], d.Y[sel3],50)
plt.xlabel('X')
plt.ylabel('Y')
plt.colorbar()

plt.show()

In [ ]:
fig = plt.figure(figsize = (11,13))

ax2 = fig.add_subplot(3,2,1)
plt.hist2d(d.R[sel3], d.S2e[sel3],[100,100], [[0,80],[2500, 7000]])
plt.xlabel('R')
plt.ylabel('S2e')
plt.colorbar()

plt.show()

In [ ]:
8500*6000/12000

## Lifetime:

In [ ]:
Zrange  = 50,  300
Znbins  =  50
Zbins = np.linspace(*Zrange, Znbins + 1)

In [ ]:
fig = plt.figure(figsize = (8,5))

S2Erange            =   2e3, 10e3
S2Enbins            = 50
S2Ebins   = np.linspace(*S2Erange   , S2Enbins + 1)

t, e, eu = fitf.profileX(d.Z[sel3], d.S2e[sel3], 50)
plt.hist2d(d.Z[sel3],d.S2e[sel3] , (np.linspace(0, d.Z[sel3].max(), 50), S2Ebins), cmin=1)
plt.errorbar(t, e, eu, fmt="kp")
plt.colorbar().set_label("# events")
plt.xlabel('Z')
plt.ylabel('S2e')

plt.show()

### Drift velocity fit:

In [ ]:
fig = plt.figure(figsize = (11,13))

ax2 = fig.add_subplot(3,2,3)
plt.hist(d.Z[sel],100 ,[310,350])
plt.yscale('log', nonposy='clip')
plt.xlabel('Z')

plt.show()